DECK


In [1]:
import operator
import pandas as pd
import numpy as np

#black jack for 6-decks
#Initializing a deck
class Deck:
    def __init__(self):
        deck = []
        for pack in range(2, 10):
            deck =deck+ [str(pack)] * 4 * 6         #4 suits and 6 decks for 2-9
        deck += ['10'] * (4+4+4+4) * 6              #K,J,Q,10(everyones value is 10)=10*(4+4+4+4) fro 6 decks
        deck+=['A'] * 4 * 6                         #4-A's for 6 decks
        self.deck = deck
    def draw(self):                                  #To retrieve a new card
        return self.deck.pop(0)
    def deck_size(self):
        return len(self.deck)                        #No.of Cards in the deck at the moment
    def riffle(self):                                #riffle shuffling a deck 
        x = np.random.randint(10, 25)
        for rif in range(x):
            np.random.shuffle(self.deck)


STATE


In [2]:
#Defining types for state and we only consider cards remaining and card states on table
#Update states based on actions
class State:
    def __init__(self):
        hard_hands=[]
        soft_hands=[]
        pair_hands=[]
        dealer_hands=[]
        for i in range(2,22):
          if i>4 and i<22:
            hard_hands.append((str(i),))              #hard hands #the absence of an Ace
          if i>=2 and i<10:
            soft_hands.append(('A',str(i)))           #soft hands #whenever you have an ace in your hand that can be counted as eleven
          if i>=2 and i<11:
            pair_hands.append((str(i),str(i)))        #pair hands #identical cards
          if i>=2 and i<=10:
            dealer_hands.append((str(i),))            #dealers hands 
        pair_hands+=[('A','A')]
        dealer_hands+=[('A',)]
        player_hands = hard_hands + soft_hands + pair_hands
        self.dealer_hands = dealer_hands        
        self.player_hands = player_hands
        black_jack_hands = [('A', '10')] 
        # Initializing available states of cards

        state_of_cards = []
        for i in player_hands:
            for j in dealer_hands:
                state_of_cards.append((i, j))
        state_of_cards += ['Terminal']
        self.state_of_cards = state_of_cards

    def possible_sums(self, hands):                        # possible sums for the sets of cards
        possibilities = [0]
        for hand in hands:
            if hand != 'A':                                #Value of A can be 1 or 11
                possibilities = [item + int(hand) for item in possibilities]
            else:
                possibilities = [item + 1 for item in possibilities] + [item + 11 for item in possibilities]
        return possibilities

#we also need to update the state of the card which is on the table
#if the player's hand is ('A', '2') and draws '3', then the state should update to('A', '5')
    def update_state_of_card(self, state_of_card, newcard):
        player = state_of_card[0]
        dealer = state_of_card[1]
        verify = []
        for item in self.possible_sums(player + (newcard, )):                   #verifying if it busts
          if item <= 21:
            verify.append(item)
        if not verify:
            return 'Terminal'

        if newcard != 'A':                                                      #If newcard is not 'A'
            if player == ('A', 'A'):
                if newcard == '10':
                    return (('12', ), dealer)
                elif newcard == '9':
                    return (('21', ), dealer)
                else:
                    instant_sum = int(newcard) + 1
                    return (('A', str(instant_sum)), dealer)

            elif 'A' in player:
                instant_sum = int(newcard) + int(player[1])
                if instant_sum == 10 or instant_sum >= 20:
                    return (('21', ), dealer)
                elif instant_sum > 10:
                    return ((str(instant_sum+1),), dealer)
                else:
                    return (('A', str(instant_sum)), dealer)

            else:
                instant_sum = sum([int(item) for item in player]) + int(newcard)
                if instant_sum == 21:
                    return (('21', ), dealer)
                else:
                    return ((str(instant_sum), ), dealer)
        
        else:                                                                   #If newcard is 'A'
            if player == ('A', 'A'):
                return (('A', '2'), dealer)

            elif 'A' in player:
                if player[1] == '9':
                    return (('21', ), dealer)
                elif player[1] == '10':
                    return (('12', ), dealer)
                else:
                    instant_sum = int(player[1]) + 1
                    return (('A', str(instant_sum)), dealer)

            else:
                s=[]
                for item in player:
                  s.append(int(item))
                instant_sum=sum(s)
                if instant_sum >= 20 or instant_sum == 10:
                    return (('21', ), dealer)
                if instant_sum > 10:
                    return ((str(instant_sum+1), ), dealer)
                else:
                    return (('A', str(instant_sum)), dealer)


CARD COUNTING

In [3]:
#Defining Card Counting Class
class Card_Counting:
    def __init__(self):                                                         #define deck states
      self.state_of_deck = 0
    def update_state_of_deck(self, newcard):                                    #Based on Card Counting lets update state of deck

#--------------------HI LOW CARD COUNTING----------------------
        """ 
        if newcard in ['2', '3', '4', '5', '6']:
            self.state_of_deck -= 1
        elif newcard in ['10', 'A']:
            self.state_of_deck += 1
        return
        """   
#--------------------OMEGA-II CARD COUNTING---------------------

        if newcard in ['4', '5', '6']:
            self.state_of_deck += 2
        elif newcard in ['2', '3', '7']:
            self.state_of_deck += 1
        elif newcard in ['10']:
            self.state_of_deck -= 2
        elif newcard in ['9']:
            self.state_of_deck -= 1

TRAINING

In [4]:
#Q-learning algorithm
class Train:
    def __init__(self, Alpha, Gamma):
        self.Alpha=Alpha                                                        # Learning Rate
        self.Gamma=Gamma
        self.state = State()
        self.Card_Counting=Card_Counting()
        self.deck = Deck()
        self.deck.riffle()
        self.Q=np.matrix(np.zeros([len(self.state.state_of_cards)*1801, 4]))

# return available actions based on the current state_of_card and action
# 0 for Stand
# 1 for HIT
# 2 for DoubleDown
# 3 for Split
    def possible_actions(self, state_of_card, action):                          # Also considering Split and DoubleDown are allowed
        if action == 'initial' or action == 'Split':
            if state_of_card == 'Terminal':
                x=[0]
                return x
            elif len(state_of_card[0]) == 2 and state_of_card[0][0] == state_of_card[0][1] :
                x=[0, 1, 2, 3]
                return x
            else:
                x=[0, 1, 2]
                return x
        elif action == 'Hit':                                                   # After Hitting Split and DoubleDown are not allowed
            if state_of_card[0][0] == '21':
                x=[0]
                return x
            else:
                x=[0, 1]
                return x
                                                                                # possible sums for the sets of cards
    def possible_sums(self, hands):
        possibilities = [0]
        for hand in hands:
            if hand != 'A':
                possibilities = [item + int(hand) for item in possibilities]
            else:
                possibilities = [item + 1 for item in possibilities] + [item + 11 for item in possibilities]
        return possibilities

                                                                                # For given card state returning index of row of the Q Matrix 
    def state_to_row(self, state_of_card):

#state_of_deck = int(state_of_deck / card_numbers * 100)
#0th Row When state_of_deck equal to -900
#900th Row When state_of_deck equal to 0
#1800th Row When state_of_deck equal to 900 

        state_of_deck = int(self.Card_Counting.state_of_deck / len(self.deck.deck) * 300)
        if state_of_card == 'Terminal':
            return (self.Card_Counting.state_of_deck+901) * len(self.state.state_of_cards)-1
        else:
            player = state_of_card[0]
            dealer = state_of_card[1]
            return (self.Card_Counting.state_of_deck+900) * len(self.state.state_of_cards) + self.state.dealer_hands.index(dealer) +\
        self.state.player_hands.index(player)*len(self.state.dealer_hands)

                                                                                # return immediate reward when the player Stands
                                                                                # update Q-matrix based on the immediate reward
    def Stand_result(self, state_of_card, hidden_dealer):
        ## find the player's maximum sum
        player = state_of_card[0]
        player_cumulative = [s for s in self.possible_sums(player) if s <= 21]
        max_max_player_sum = max(player_cumulative)

        dealer = state_of_card[1] + (hidden_dealer, )                           # reveal the hidden dealer card
        dealer_cumulative = [s for s in self.possible_sums(dealer) if s <= 21]
        reward = 2                                                              # initializing reward

                                                                                # dealer keeps Hitting until going bust
        while dealer_cumulative:
            if max(dealer_cumulative) > max_max_player_sum:                     # dealer wins!
                reward = 0
                break
            if max(dealer_cumulative) >= 17:
                if max(dealer_cumulative) == max_max_player_sum:                # draw!
                    reward = 1
                    break
                else:
                    reward = 2                                                  # player wins!
                    break
                                                                                # dealer draws a new card and update states
            newcard = self.deck.draw()
            self.Card_Counting.update_state_of_deck(newcard)
            dealer_cumulative = [s for s in self.possible_sums(dealer_cumulative + [newcard]) if s <= 21]

                                                                                # update Q
        row = self.state_to_row(state_of_card)
        nextrow = self.state_to_row('Terminal')
        self.Q[row, 0] = (1-self.Alpha)*self.Q[row, 0] + self.Alpha*(reward+self.Gamma*self.Q[nextrow, 0])
        return reward
                                                                                # return immediate reward when the player DoubleDowns
                                                                                # update Q-matrix based on the immediate reward
    def DoubleDown_result(self, state_of_card, hidden_dealer):
        player = state_of_card[0]
                                                                                # Player draws a new card
        newcard = self.deck.draw()
        self.Card_Counting.update_state_of_deck(newcard)
        player_cumulative = [s for s in self.possible_sums(player + (newcard, )) if s <= 21]
        if not player_cumulative:                                               # player busts
            row = self.state_to_row(state_of_card)
            nextrow = self.state_to_row('Terminal')
            self.Q[row, 2] = (1-self.Alpha)*self.Q[row, 2] + self.Alpha*(-1+self.Gamma*self.Q[nextrow, 0])
            return -1

                                                                                # find the maximum sum for the player
        max_max_player_sum = max(player_cumulative)
        dealer = state_of_card[1] + (hidden_dealer, )
        dealer_cumulative = [s for s in self.possible_sums(dealer) if s <= 21]
        reward = 3                                                              # initialize a reward

                                                                                # dealer Hits until bust
        while dealer_cumulative:
            if max(dealer_cumulative) > max_max_player_sum: # dealer wins!
                reward = -1
                break
            if max(dealer_cumulative) >= 17:
                if max(dealer_cumulative) == max_max_player_sum: # draw!
                    reward = 1
                    break
                else:
                    reward = 3                                                  # player wins!
                    break
                                                                    
            newcard = self.deck.draw()                                          #dealer drawing a new card
            self.Card_Counting.update_state_of_deck(newcard)
            dealer_cumulative = [s for s in self.possible_sums(dealer_cumulative + [newcard]) if s <= 21]

                                                                                # updating Q
        row = self.state_to_row(state_of_card)
        nextrow = self.state_to_row('Terminal')
        self.Q[row, 2] = (1-self.Alpha)*self.Q[row, 2] + self.Alpha*(reward+self.Gamma*self.Q[nextrow, 0])
        return reward

                                                                                #return gainsafter one game
    def game(self, state_of_card=(), hidden_dealer=False, game_stage='initial'):
        if game_stage == 'initial': # game starts (default)
                                                                                 # draw 2 new cards
            player = tuple(self.deck.draw() for i in range(2))
            dealer = tuple(self.deck.draw() for i in range(2))
                                                                                # update state of the deck
            for item in player:
                self.Card_Counting.update_state_of_deck(item)
            for item in dealer:
                self.Card_Counting.update_state_of_deck(item)
                                                                                # second card is upside-down
            open_dealer = dealer[0]
            hidden_dealer = dealer[1]

            if set(player) == set(['10', 'A']):                                 # Black Jack
                return 2.5

            if 'A' in player:
                index_A = player.index('A')
                player = (player[index_A], player[index_A^1])
            elif player[0] != player[1]:
                player = (str(sum([int(p) for p in player])), )
            state_of_card = (player, (open_dealer, ))
                                                                                # Continuing game after the player Splits
        elif game_stage == 'Split':
            newcard = self.deck.draw()
            self.Card_Counting.update_state_of_deck(newcard)
                                                                                # 'A' exists in the player's hand
            if state_of_card[0][0] == 'A':
                if newcard == '10':
                    return 2.5
                state_of_card = (('A', newcard), state_of_card[1])
                                                                                # 'A' does not exist in the player's hand
            else:
                if newcard == 'A':
                    if state_of_card[0][0] == '10':
                        return 2.5
                    state_of_card = (('A', state_of_card[0][0]), state_of_card[1])
                else:
                    if state_of_card[0][0] == newcard:
                        state_of_card = ((newcard, newcard), state_of_card[1])
                    else:
                        state_of_card = ((str(int(newcard)+int(state_of_card[0][0])), ), state_of_card[1])
                                                                                # Continuing Game after the player Hits
        elif game_stage == 'Hit':
                                                                                # draw a new card
            row = self.state_to_row(state_of_card)
            newcard = self.deck.draw()
                                                                                # update deck and card states
            self.Card_Counting.update_state_of_deck(newcard)
            state_of_card = self.state.update_state_of_card(state_of_card, newcard)
            nextrow = self.state_to_row(state_of_card)
            if state_of_card == 'Terminal':
                self.Q[row, 1] = (1-self.Alpha)*self.Q[row, 1] + self.Alpha*(0+self.Gamma*self.Q[nextrow, 0])
                return 0
                                                                                # update Q
            further_actions = self.possible_actions(state_of_card, game_stage)
            max_index = 0
            for item in further_actions:
                if self.Q[nextrow, item] > self.Q[nextrow, max_index]:
                    max_index = item

            self.Q[row, 1] = (1-self.Alpha)*self.Q[row, 1] + self.Alpha*(0+self.Gamma*self.Q[nextrow, max_index])
                                                                                # possible actions based on state_of_card and game_stage
        actions = self.possible_actions(state_of_card, game_stage)
        action = np.random.choice(actions)

        if action == 0:                                                         # Stand
            return self.Stand_result(state_of_card, hidden_dealer)
        elif action == 1:                                                       # Hit
            return self.game(state_of_card=state_of_card, hidden_dealer=hidden_dealer, game_stage='Hit')
        elif action == 2:                                                       # double down
            return self.DoubleDown_result(state_of_card, hidden_dealer)
        else:                                                                   # Split
            row = self.state_to_row(state_of_card)
            state_of_card = ((state_of_card[0][0], ), state_of_card[1])
            reward1 = self.game(state_of_card=state_of_card, hidden_dealer=hidden_dealer, game_stage='Split')
            reward2 = self.game(state_of_card=state_of_card, hidden_dealer=hidden_dealer, game_stage='Split')
                                                                                # update Q
            nextrow = self.state_to_row('Terminal')
            self.Q[row, 3] = (1-self.Alpha)*self.Q[row, 3] + self.Alpha*((reward1+reward2)+self.Gamma*self.Q[nextrow, 0])
            return reward1 + reward2

                                                                                # Train the model using Q-learning n times
    def Train(self, n):
        for i in range(n):
                                                                                # if remaining cards < 40, dealer uses new card set
            if len(self.deck.deck) < 40:
                self.deck = Deck()
                self.deck.riffle()
                self.Card_Counting.state_of_deck = 0
            self.game()


BACKTESTING BASED ON TRAINING

In [5]:
# BackTesting based on the Trained matrix Q
class Backtest:
                                                                                # initialize deck, state, and Q
    def __init__(self, Q):
        self.state=State()
        self.deck=Deck()
        self.Card_Counting=Card_Counting()
        self.deck.riffle()
        self.Q=Q

    def possible_actions(self, state_of_card, action):                          # Also considering Split and DoubleDown are allowed
        if action == 'initial' or action == 'Split':
            if state_of_card == 'Terminal':
                x=[0]
                return x
            elif len(state_of_card[0]) == 2 and state_of_card[0][0] == state_of_card[0][1] :
                x=[0, 1, 2, 3]
                return x
            else:
                x=[0, 1, 2]
                return x
        elif action == 'Hit':                                                   # After Hitting Split and DoubleDown are not allowed
            if state_of_card[0][0] == '21':
                x=[0]
                return x
            else:
                x=[0, 1]
                return x
                                                                                # return row index of the Q Matrix for the given card state
    def state_to_row(self, state_of_card):
        if state_of_card == 'Terminal':
            return (self.Card_Counting.state_of_deck+901) * len(self.state.state_of_cards)-1
        else:
            player = state_of_card[0]
            dealer = state_of_card[1]
            return (self.Card_Counting.state_of_deck+900) * len(self.state.state_of_cards) +\
        len(self.state.dealer_hands) * self.state.player_hands.index(player) + self.state.dealer_hands.index(dealer)

    ## possible sums for the sets of cards
    def possible_sums(self, hands):
        possibilities = [0]
        for hand in hands:
            if hand != 'A':
                possibilities = [item + int(hand) for item in possibilities]
            else:
                possibilities = [item + 1 for item in possibilities] + [item + 11 for item in possibilities]
        return possibilities

                                                                                # return the reward when the player Stands
    def Stand_result(self, state_of_card, hidden_dealer):
                                                                                # find the player's maximum sum
        player = state_of_card[0]
        player_cumulative = [s for s in self.possible_sums(player) if s <= 21]
        max_player_sum = max(player_cumulative)
                                                                                # reveal the upside down card
        dealer = state_of_card[1] + (hidden_dealer, )
        dealer_cumulative = [s for s in self.possible_sums(dealer) if s <= 21]
                                                                                # dealer keeps Hitting until going bust
        while dealer_cumulative:
            if max(dealer_cumulative) > max_player_sum:                         # dealer wins!
                return 0
            if max(dealer_cumulative) >= 17:
                if max(dealer_cumulative) == max_player_sum:                    # draw!
                    return 1
                else:
                    return 2                                                    # player wins!

                                                                                # dealer draws a new card
            newcard = self.deck.draw()
            self.Card_Counting.update_state_of_deck(newcard)
            dealer_cumulative = [s for s in self.possible_sums(dealer_cumulative + [newcard]) if s <= 21]
        return 2

                                                                                # return immediate return when the player DoubleDowns
    def DoubleDown_result(self, state_of_card, hidden_dealer):
        player = state_of_card[0]
                                                                                # draw a new card
        newcard = self.deck.draw()
        self.Card_Counting.update_state_of_deck(newcard)
        player_cumulative = [s for s in self.possible_sums(player + (newcard, )) if s <= 21]
        if not player_cumulative:                                               # player busts
            return -1
                                                                                # find the maximum sum of the player
        max_player_sum = max(player_cumulative)
        dealer = state_of_card[1] + (hidden_dealer, )
        dealer_cumulative = [s for s in self.possible_sums(dealer) if s <= 21]
                                                                                # dealer keeps Hitting until going bust
        while dealer_cumulative:
            if max(dealer_cumulative) > max_player_sum:                         # dealer wins!
                return -1
            if max(dealer_cumulative) >= 17:
                if max(dealer_cumulative) == max_player_sum:                    # draw!
                    return 1
                else:
                    return 3                                                    # player wins!
                                                                                # dealer draws a new card
            newcard = self.deck.draw()
            self.Card_Counting.update_state_of_deck(newcard)
            dealer_cumulative = [s for s in self.possible_sums(dealer_cumulative + [newcard]) if s <= 21]
        return 3

    # play one game and return gains
    def game(self, state_of_card=(), hidden_dealer=False, game_stage='initial'):
        if game_stage == 'initial':                                             # game starts
                                                                                # draw 2 new cards
            player = tuple(self.deck.draw() for _ in range(2))
            dealer = tuple(self.deck.draw() for _ in range(2))
                                                                                # update the state of the deck
            for p in player:
                self.Card_Counting.update_state_of_deck(p)
            for d in dealer:
                self.Card_Counting.update_state_of_deck(d)
                                                                                # the second card is upside-down
            open_dealer = dealer[0]
            hidden_dealer = dealer[1]
            if set(player) == set(['10', 'A']):                                 # Blackjack!
                return 2.5
            if 'A' in player:
                index_A = player.index('A')
                player = (player[index_A], player[index_A^1])
            elif player[0] != player[1]:
                player = (str(sum([int(p) for p in player])), )
            state_of_card = (player, (open_dealer, ))
                                                                                # Game continues after the player Splits
        elif game_stage == 'Split':
            newcard = self.deck.draw()
            self.Card_Counting.update_state_of_deck(newcard)
                                                                                # 'A' exists in the player's hand
            if state_of_card[0][0] == 'A':
                if newcard == '10':
                    return 2.5
                state_of_card = (('A', newcard), state_of_card[1])
                                                                                # 'A' does not exist in the player's hand
            else:
                if newcard == 'A':
                    if state_of_card[0][0] == '10':
                        return 2.5
                    state_of_card = (('A', state_of_card[0][0]), state_of_card[1])
                else:
                    if state_of_card[0][0] == newcard:
                        state_of_card = ((newcard, newcard), state_of_card[1])
                    else:
                        state_of_card = ((str(int(newcard)+int(state_of_card[0][0])), ), state_of_card[1])
                                                                                # Game continues after the player Hits
        elif game_stage == 'Hit':
                                                                                # draw a new card
            newcard = self.deck.draw()
                                                                                # update deck and card states
            self.Card_Counting.update_state_of_deck(newcard)
            state_of_card = self.state.update_state_of_card(state_of_card, newcard)
            if state_of_card == 'Terminal':
                return 0

        row = self.state_to_row(state_of_card)
        actions = self.possible_actions(state_of_card, game_stage)
        d = {action: self.Q[row, action] for action in actions}
        action = sorted(d.items(), key=operator.itemgetter(1))[::-1][0][0]

        if action == 0:                                                         # Stand
            return self.Stand_result(state_of_card, hidden_dealer)
        elif action == 1:                                                       # Hit
            return self.game(state_of_card=state_of_card, hidden_dealer=hidden_dealer, game_stage='Hit')
        elif action == 2:                                                       # double down
            return self.DoubleDown_result(state_of_card, hidden_dealer)
        else:                                                                   # Split
            state_of_card = ((state_of_card[0][0], ), state_of_card[1])
            return1 = self.game(state_of_card=state_of_card, hidden_dealer=hidden_dealer, game_stage='Split')
            return2 = self.game(state_of_card=state_of_card, hidden_dealer=hidden_dealer, game_stage='Split')
            return return1 + return2

# backtesting the model and return the gains for n games
    def backtest(self, n):
        gains = [0]
        for _ in range(n):
            if len(self.deck.deck) < 40:
                self.deck=Deck()
                self.deck.riffle()
                self.Card_Counting.state_of_deck = 0
            gains.append(self.game())
        return gains


In [ ]:
# Train 6-deck blackjack game
#Learning Rate (Alpha) = 0.1
#discount (Gamma) = 1
t=Train(0.1, 1)
t.Train(400000)
#backtesting the blackjack game 20000 times based on the previous Training result
backtest = Backtest(t.Q)
payoff = backtest.backtest(20000)
# calculate the accumulated payoff
total_payoff = [payoff[0]]
for i in range(1, 20000):
    total_payoff.append(total_payoff[i-1] + payoff[i])
# find the accumulated winning odds
winning_odds = [total_payoff[i]/(2*(i+1)) for i in range(20000)]
print("Winning Odds after playing 20000 games",winning_odds[-1])

Two Rule Variations have been made
1. Using only one deck of cards to play black Jack
Winning Odds after playing 20000 games with only one deck : 49.8%

2. Dealer need not hit until  he gets to 17
Winning Odds after playing 20000 games : 73.29%